In [1]:
%matplotlib inline
import torch
import torchvision
from torchvision.transforms import v2 as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import kagglehub

In [ ]:
path = kagglehub.dataset_download("gergvincze/simple-hand-drawn-and-digitized-images")

print("Path to dataset files:", path)

In [ ]:
mean = np.array([0.4914, 0.4822, 0.4465])
std = np.array([0.2023, 0.1994, 0.2010])

train_transforms = transforms.Compose([
    transforms.ToImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(3),
    transforms.RandomPerspective(0.1),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean, std),
])

val_transforms = transforms.Compose([
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean, std)
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=val_transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                      a   shuffle=False, num_workers=2)

rawset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')